In [ ]:
import json

with open('./res/reviews.json', 'r', encoding='utf-8') as f:
    review_list = json.load(f)

print(review_list)



In [4]:
good_cnt, bad_cnt = 0,0
for review in review_list:
    if review['stars'] == 5:
        good_cnt += 1
    else:
        bad_cnt += 1

good_cnt, bad_cnt

(172, 48)

In [8]:
reviews_good, reviews_bad = [], []

for review in review_list:
    if review['stars'] == 5:
        reviews_good.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')
    else:
        reviews_bad.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')

reviews_bad[:3]

['[REVIEW_START]주차불편 도보이용 외에는 다 좋았어요[REVIEW_END]',
 '[REVIEW_START]좋았어요 무냐히 사용합니다[REVIEW_END]',
 '[REVIEW_START]1. 신라스테이는 여름에 가지 않는걸 추천드립니다. 중앙제어 냉방 시스템이기 때문에, 묵었던 날이 폭염이었음에도 불구하고 프론트에 연락해도 아무 것도 해줄 수 없다는 말만 들었습니다. 요즘 시대에 아직도 중앙제어 온도 기능이 있다는게 놀랍습니다. 그래도 친절한 직원분들 덕분에 선풍기 하나는 받아서 잠은 잤습니다. 이건 서비스가 아니라 구조적인 문제라고 생각되네요.\n\n2. 핸드폰이 침대 옆에 떨어져 줍다가 다리가 많은 돈벌레를 보고 놀랐습니다. 저는 그나마 벌레를 잘 잡는 편이라 그 자리에서 잡고 다음 날 클레임도 안하고 대수롭지 않게 넘어갔지만 벌레를 혐오하거나 무서워하는 분들이 봤어도 이렇게 조용히 넘어갔을까 하는 생각이 듭니다.\n\n그동안 신라스테이 광화문, 서대문, 구로, 삼성, 마포 등 서울 내 지점들을 이용한 횟수만 30회 가까이 되는데 앞으로는 조금 더 투자해서 다른 곳에 묵을 예정입니다. 제가 이 정도 비용을 지불하고, 벌레 안 나오는 시원한 곳에서 편하게 묵으려고 한게 과한 욕심이었을까요...?[REVIEW_END]']

In [26]:
import datetime
from dateutil import parser

def preprocess_reviews(path='./res/reviews.json'):
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)
    
    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    for review in review_list:
        review_date_str = review['date']
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            review_date = current_date
        
        if review_date < date_boundary:
            continue
        if review['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')
    
    review_good_text = '\n'.join(reviews_good)
    review_bad_text = '\n'.join(reviews_bad)

    return review_good_text, review_bad_text

good, bad = preprocess_reviews()
good[:100]
len(good)

5551

In [12]:
def pairwise_eval(reviews, answer_a, answer_b):
    eval_prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the Korean summaries provided by two
AI assistants to the set of user reviews on accommodations displayed below. You should choose the assistant that
follows the user’s instructions and answers the user’s question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.
[User Reviews]
{reviews}
[The Start of Assistant A’s Answer]
{answer_a}
[The End of Assistant A’s Answer]
[The Start of Assistant B’s Answer]
{answer_b}
[The End of Assistant B’s Answer]"""
    
    completion = client.chat.completions.create(
        model='gpt-4o-2024-05-13',
        messages=[{'role': 'user', 'content': eval_prompt}],
        temperature=0.0
    )

    return completion

In [13]:
PROMPT_BASELINE = f"""아래 숙소에 대해서 5문장 내로 요약해줘:"""

In [15]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()


True

In [16]:
reviews, _ = preprocess_reviews(path="./res/reviews.json")

client = OpenAI(api_key=os.getenv('OPNEAI_API_KEY'))

def summarize(reviews, prompt, temperature = 0.0, model = 'gpt-3.5-turbo-0125'):
    prompt = prompt +'\n\n' +reviews

    completion = client.chat.completions.create(
        model = model,
        messages = [{'role': 'user', 'content': prompt}],
        temperature=temperature
    )

    return completion

print(summarize(reviews, PROMPT_BASELINE).choices[0].message.content)

깨끗하고 편안한 숙소, 친절한 직원, 교통이 편리한 위치, 조식이 맛있는 신라스테이 숙소입니다.


In [17]:
eval_count = 10

summaries_baseline = [summarize(reviews, PROMPT_BASELINE, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
summaries_baseline

['신라스테이 공덕역 숙소는 깨끗하고 친절한 서비스로 잘 알려져 있습니다. 도심에 위치한 숙소로 출장이나 여행에 이용하기 좋습니다. 시설과 침구는 만족스럽지만 몇 가지 관리가 필요한 부분도 있습니다. 또한 가격 대비 좋은 편이지만 개선이 필요한 부분도 있습니다.',
 '이 숙소는 청결하고 깨끗하며 직원 분들이 친절한 호텔로, 교통이 편리하고 다양한 음식점이 주변에 있어 출장이나 여행에 적합합니다. 하지만 몇몇 객실에서는 관리가 덜 된 부분이 있을 수 있으며, 에어컨이 전체적으로 조절되어 따뜻한 날씨에는 조절이 어려울 수 있습니다.',
 '신라스테이는 깨끗하고 친절한 직원이 인상적인 곳이며 위치가 편리하다.하지만 조금 노후화된 시설들이 있고 가끔 소음이 있을 수 있으며 가격은 저렴하지 않다.',
 '신라스테이 숙소는 교통이 편리하고 깨끗한 곳으로 휴식하기 좋다. 직원의 친절함이 인상적이며, 재방문을 고려했다. 특히 침구는 편안하고 조식도 맛있었지만, 조금 노후화된 시설이 아쉬웠다. 복도 소음 등 몇 가지 부족한 부분이 있었지만, 전반적으로 만족스러운 숙소라는 평가가 많았다.',
 '깨끗하고 친절한 직원들이 있는 숙소로 다음에도 이용할 예정이며, 잘 쉬고 간 여행객들을 위한 편안한 시설을 제공하고 있습니다. 위치가 편리하고 깔끔한 분위기를 자랑하지만 몇몇 객실은 관리가 미흡한 면이 있다는 후기도 있습니다.',
 '깨끗하고 편안한 숙소로, 교통이 편리하며 직원들도 친절하다. 다만 일부 시설 노후화 등 아쉬운 점 있음.',
 '공덕역 근처 숙소는 깨끗하고 좋다. 항상 만족스러운 서비스를 제공하며, 위치가 편리하고 직원들도 친절하다. 다만 시설의 일부는 노후화되어 있어 업그레이드가 필요할 수 있다.',
 '쾌적하고 깨끗한 숙소로, 위치와 직원 친절이 좋다. 하지만 몇몇 방음과 시설 노후화에 대한 언급도 있다.',
 '깨끗하고 편안한 신라스테이, 지하철 역 근처에 위치해 교통이 편리하고 친절한 직원들이 인상적이며 재방문을 생각하게 하는 숙소입니다.',
 '신라스테이는 공

In [18]:
summary_real_20241003 = '교통이 편리한 위치에 있으며, 청결한 상태를 유지하고 있어 많은 후기가 긍정적입니다. 객실은 넷플릭스 시청을 위한 기기 대여가 필요하지만 전반적으로 깔끔하고 편안한 분위기를 제공합니다. 직원들의 친절한 응대가 인상적이며, 조식과 디너 서비스 역시 우수하다는 평이 있습니다. 다만, 방음과 시설 노후화 관련 아쉬운 점이 언급되기도 했습니다. 종합적으로 접근성과 서비스가 우수한 조건을 갖춘 숙소입니다.'

In [21]:
from tqdm import tqdm

def pairwise_eval_batch(reviews, answers_a, answers_b):
    a_cnt, b_cnt, draw_cnt = 0,0,0
    for i in tqdm(range(len(answers_a))):
        completion = pairwise_eval(reviews, answers_a[i], answers_b[i])
        verdict_text = completion.choices[0].message.content

        if '[[A]]' in verdict_text:
            a_cnt += 1
        elif '[[B]]' in verdict_text:
            b_cnt += 1        
        elif '[[C]]' in verdict_text:
            draw_cnt += 1
        else:
            print("Evaluation Error")

    return a_cnt, b_cnt, draw_cnt


wins, loses, ties = pairwise_eval_batch(reviews, summaries_baseline, [summary_real_20241003]*eval_count)
print(f"Wins: {wins}, Loses: {loses}, Ties: {ties}")


100%|██████████| 10/10 [00:30<00:00,  3.01s/it]

Wins: 1, Loses: 9, Ties: 0


### 고도화: 조건 명시

In [22]:
prompt = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20241003 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:31<00:00,  3.15s/it]

Wins: 1, Losses: 9, Ties: 0


# 고도화2: 

In [29]:
import datetime
from dateutil import parser

def preprocess_reviews(path='./res/reviews.json'):
    with open(path, 'r', encoding='utf-8') as f:
        review_list = json.load(f)
    
    reviews_good, reviews_bad = [], []

    current_date = datetime.datetime.now()
    date_boundary = current_date - datetime.timedelta(days=6*30)

    for review in review_list:
        review_date_str = review['date']
        try:
            review_date = parser.parse(review_date_str)
        except (ValueError, TypeError):
            review_date = current_date
        
        if review_date < date_boundary:
            continue
        if len(review['review']) < 30:
            continue
        
        if review['stars'] == 5:
            reviews_good.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')
        else:
            reviews_bad.append('[REVIEW_START]' + review['review'] + '[REVIEW_END]')
    
    review_good_text = '\n'.join(reviews_good)
    review_bad_text = '\n'.join(reviews_bad)

    return review_good_text, review_bad_text

good, bad = preprocess_reviews()
good[:100]
len(good)

2808

In [24]:
eval_count = 10
summaries = [summarize(reviews, prompt, temperature=1.0).choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20241003 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:34<00:00,  3.41s/it]

Wins: 0, Losses: 10, Ties: 0


# Few-shot

In [27]:
reviews_1shot, _ = preprocess_reviews(path="./res/shillastay_Seodaemun.json")
summary_1shot = summarize(reviews_1shot, prompt, temperature = 0.0, model = 'gpt-4o-2024-05-13').choices[0].message.content

In [28]:
prompt_1shot = f"""당신은 요약 전문가입니다. 사용자 숙소 리뷰들이 주어졌을 때 요약하는 것이 당신의 목표입니다.

요약 결과는 다음 조건들을 충족해야 합니다:
1. 모든 문장은 항상 존댓말로 끝나야 합니다.
2. 숙소에 대해 소개하는 톤앤매너로 작성해주세요.
  2-1. 좋은 예시
    a) 전반적으로 좋은 숙소였고 방음도 괜찮았다는 평입니다.
    b) 재방문 예정이라는 평들이 존재합니다.
  2-2. 나쁜 예시
    a) 좋은 숙소였고 방음도 괜찮았습니다.
    b) 재방문 예정입니다.
3. 요약 결과는 최소 2문장, 최대 5문장 사이로 작성해주세요.

다음은 리뷰들과 요약 예시입니다.
예시 리뷰들:
{reviews_1shot}
예시 요약 결과:
{summary_1shot}
    
아래 숙소 리뷰들에 대해 요약해주세요:"""

summaries = [summarize(reviews, prompt, temperature=1.0, model='gpt-3.5-turbo-0125').choices[0].message.content for _ in range(eval_count)]
wins, losses, ties = pairwise_eval_batch(reviews, summaries, [summary_real_20241003 for _ in range(len(summaries))])
print(f'Wins: {wins}, Losses: {losses}, Ties: {ties}')

100%|██████████| 10/10 [00:29<00:00,  2.93s/it]

Wins: 0, Losses: 10, Ties: 0
